In [1]:
import torch
import torchvision
import PIL
import os
from matplotlib import pyplot as plt
import cv2
import random
import warnings
import timm
from torch import nn
import numpy as np
from tqdm import tqdm_notebook
from torchvision.models import mobilenet_v3_small
from torchsummary import summary
import time

warnings.filterwarnings('ignore')

device='cuda' if torch.cuda.is_available() else 'cpu'

print(f'Using: {device}')


Using: cuda


In [2]:
class dataset():
    def __init__(self,src,des):
        self.des=des
        self.src=src
        self.images=os.listdir(src)
        
    def __len__(self):
        return len(self.images)

    def apply_mosaic(self,image, top_left_x, top_left_y, width, height, mosaic_size):

        roi = image[top_left_y:top_left_y+height, top_left_x:top_left_x+width]
    
        roi_small = cv2.resize(roi, (mosaic_size, mosaic_size), interpolation=cv2.INTER_LINEAR)
    
        mosaic = cv2.resize(roi_small, (width, height), interpolation=cv2.INTER_NEAREST)
        
        image[top_left_y:top_left_y+height, top_left_x:top_left_x+width] = mosaic
        return image


    def make_ds(self):
        count=0
        for idx in range(len(self.images)):
            
            img=cv2.imread(self.src+"/"+self.images[idx])

            original=np.copy(img)
          
            img_height, img_width = img.shape[:2]
    
            # Define the size of the mosaic area
            mosaic_width = random.randint(50,150)
            mosaic_height = random.randint(50,150)
            mosaic_size = random.randint(5,10)  # Size of each mosaic block
            
            # Randomly pick a spot
            top_left_x = random.randint(0, img_width - mosaic_width)
            top_left_y = random.randint(0, img_height - mosaic_height)
            
            # Apply the mosaic effect
            output_image = self.apply_mosaic(img, top_left_x, top_left_y, mosaic_width, mosaic_height, mosaic_size)

            #output_image = self.transforms(output_image.permute(2,0,1)).permute(1,2,0)
            #original=self.transforms(torch.from_numpy(original).permute(2,0,1)).permute(1,2,0)
            
            cv2.imwrite(f'{self.des}/train_A/{self.images[idx]}',original)

            cv2.imwrite(f'{self.des}/train_B/{self.images[idx]}',output_image)
            
            count+=1
            
            if(count%100==0):
                print(f'{count} conversions compelete')

In [3]:
ds=dataset('./temp/food-101/images/tuna_tartare/','./temp/food-101/images/tuna_ds/')

In [4]:
ds.make_ds()

100 conversions compelete
200 conversions compelete
300 conversions compelete
400 conversions compelete
500 conversions compelete
600 conversions compelete
700 conversions compelete
800 conversions compelete
900 conversions compelete
1000 conversions compelete
